# KDE Custom Motor Head

**Goal:** The KDE motor overheats while running. If we increase airflow over the coils, can we improve the runtime of the motor add different power settings?

For our 3D printed part, running it with a prop would tear it apart. Therefore, initial tests should be run with a prop. The main goal of a test should be to show that the rates of temperature change are reduced with the custom head.

The KDE website states that their components are designed to have a 5 mph windowflow when they are run. However, we can not measure the effect the fan's airflow provides. As a result, we should use a strict upper and lower bound on temperature.

## Test Structure

The test will consist of decreasing throttle steps with cooling in between. 
Initially, the motor will be heated to the starting temperature.
Then the motor will be run at the max throttle until the end of a time period, or the thermal safety cutoff is hit. 
The motor will sit to cool back down to the starting temperature before the next throttle setting lower than max is run.

```
Throttle:
 .----.
 |    |   .----.   
 |    |   |    |   .----.
 |    |   |    |   |    |   .----.
_|    |___|    |___|    |___|    |___

```

## Test Parameters
| Param | Value | Reason |
|---|--|---|
| **Start temp**          | 95 \*F      |
| **Safety Cutoff temp**  | 115 \*F     |
| **Max PWM**             | 2000        | 100%
| **Min PWM**             | 1200        | 20%
| **Heating Time**        | 5 mins      | pretty good at separating runaway in KDE-Levels
| **PWM Step**            | -100        | 10% throttle


In [1]:
from analysis_tools import *
import bokeh
from bokeh.plotting import *
output_notebook()

Loading BokehJS ...

In [2]:
unFilteredRuns = runsFromFolder('custom_head_testing')

In [3]:
runs = unFilteredRuns
#runs = list(filter(lambda x: '1200' in x.file, unFilteredRuns))

In [4]:
for r in runs:
    r.props["cutoffs"] = findSpikes(r, eqThrottle=[2000, 1900, 1800, 1700, 1600, 1500, 1400, 1300, 1200], minThrottle=1000)[1:]

In [5]:
for run in runs:
    testSegments = dict()
    run.props["segments"] = testSegments
    for cutoff in run.props["cutoffs"]:
        testPwm = run.data[cutoff].pwm
        i = cutoff
        while i > 0 and run.data[i].pwm == testPwm:
            i -= 1
        testSegments[int(testPwm)] = (i, cutoff+1)

In [6]:
plot = figure()

for run in runs:
    pwms = list(run.props["segments"].keys())
    
    for pwm in pwms:
        if pwm == 1200:
            break
    
        color = mapPwmToColor(pwm)
        segment = run.props["segments"][pwm]
        pts = run.data[segment[0]:segment[1]]
    
        startTime = pts[0].time
    
        times = [pt.time - startTime for pt in pts]
        temps = [pt.motorTemp for pt in pts]
    
        plot.line(times, temps, color=color)
        
show(plot)    

In [11]:
plot = figure()
for run in runs:
    plot.line([x.time for x in run.data], [x.motorTemp for x in run.data])
    #plot.line([x.time for x in run.data], [x.pwm * (160 / 2000) for x in run.data])
show(plot)